# Fitting IM Lup

This code runs the MCMC simulation to calculate the best fit parameters for the disk. It uses the logprob function from logprob_parallel.py.

In [ ]:
import tempfile
import pickle
from copy import deepcopy
from pathlib import Path
from multiprocessing import Pool

import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
import emcee

from imgcube import imagecube
import dsharp_helper as dh
import dsharp_opac as opacity

from logprob_parallel import logprob

In [ ]:
from dipsy.utils import get_interfaces_from_log_cell_centers
from dipsy import get_powerlaw_dust_distribution

In [ ]:
import warnings
import disklab

au = c.au.cgs.value
M_sun = c.M_sun.cgs.value
L_sun = c.L_sun.cgs.value
R_sun = c.R_sun.cgs.value

## ALMA data 

set the disk name and get some disk properties from DSHARP

In [ ]:
disk = 'IMLup'

fname_mm_obs = dh.get_datafile(disk)
PA = dh.sources.loc[disk]['PA']
inc = dh.sources.loc[disk]['inc']
distance = dh.sources.loc[disk]['distance [pc]']

Get the observed mm profile from DSHARP and determine a radial profile with `imagecube` which is used as an input

In [ ]:
data_mm_obs = imagecube(fname_mm_obs, clip=2.5)

x_as_obs, y_obs, dy_obs = data_mm_obs.radial_profile(inc=inc, PA=PA)

profile_mm_obs = (y_obs * u.Jy / data_mm_obs.beam_area_str).cgs.value
profile_mm_obs_err = (dy_obs * u.Jy / data_mm_obs.beam_area_str).cgs.value

In [ ]:
f, ax = plt.subplots()
ax.semilogy(x_as_obs, profile_mm_obs)
ax.fill_between(x_as_obs, profile_mm_obs - profile_mm_obs_err, profile_mm_obs + profile_mm_obs_err, alpha=0.5)
ax.set_ylim(bottom=1e-16);

## Opacities

define the wavelength, size, and angle grids

In [ ]:
n_lam = 200
n_a = 15
n_theta = 100
lam_opac = np.logspace(-5, 1, n_lam)
a_opac = np.logspace(-5, 1, n_a)

if n_theta // 2 == n_theta / 2:
    n_theta += 1
    print(f'n_theta needs to be odd, will set it to {n_theta}')

calculate opacities and store them in a local file

In [ ]:
# get optical constants and material density
opac_fname = 'dustkappa_IMLUP'
opac_fname = Path(opac_fname + '.npz')
diel_const, rho_s = opacity.get_dsharp_mix()

run_opac = True

if opac_fname.is_file():
    with np.load(opac_fname) as fid:
        opac_dict = {k:v for k,v in fid.items()}
    if (
        (len(opac_dict['a']) == n_a) and
        np.allclose(opac_dict['a'], a_opac) and
        (len(opac_dict['lam']) == n_lam) and
        np.allclose(opac_dict['lam'], lam_opac) and
        (len(opac_dict['theta']) == n_theta) and
        (opac_dict['rho_s'] == rho_s)
        ):
        print(f'reading from file {opac_fname}')
        run_opac = False

if run_opac:
    # call the Mie calculation & store the opacity in a npz file
    opac_dict = opacity.get_smooth_opacities(
        a_opac,
        lam_opac,
        rho_s=rho_s,
        diel_const=diel_const,
        extrapolate_large_grains=False,
        n_angle=(n_theta + 1) //2)

    print(f'writing opacity to {opac_fname} ... ', end='', flush=True)
    opacity.write_disklab_opacity(opac_fname, opac_dict)
    print('Done!')

This part chops the very-forward scattering part of the phase function. This part is basically the same as no scattering, but are treated by the code as a scattering event. By cutting this part out of the phase function, we avoid those non-scattering scattering events. This needs to recalculate $\kappa_{sca}$ and $g$.

In [ ]:
m = 4 * np.pi / 3 * rho_s * a_opac**3

k_abs = opac_dict['k_abs']
k_sca = opac_dict['k_sca']
S1 = opac_dict['S1']
S2 = opac_dict['S2']
theta = opac_dict['theta']
g = opac_dict['g']

zscat = opacity.calculate_mueller_matrix(lam_opac, m, S1, S2, theta=theta, k_sca=k_sca)['zscat']

chopforward = 3
zscat_nochop = np.zeros((n_a, n_lam, n_theta, 6))
kscat_nochop = np.zeros((n_a, n_lam))
g_nochop = np.zeros((n_a, n_lam))

for grain in range(n_a):
    for i in range(n_lam):
        #
        # Now loop over the grain sizes
        #
        if chopforward > 0:
            iang = np.where(theta < chopforward)
            if theta[0] == 0.0:
                iiang = np.max(iang) + 1
            else:
                iiang = np.min(iang) - 1
            zscat_nochop[grain, i, :, :] = zscat[grain, i, :, :]  # Backup
            kscat_nochop[grain, i] = k_sca[grain, i]      # Backup
            g_nochop[grain, i] = g[grain, i]
            zscat[grain, i, iang, :] = zscat[grain, i, iiang, :]
            mu = np.cos(theta * np.pi / 180.)
            dmu = np.abs(mu[1:n_theta] - mu[0:(n_theta - 1)])
            zav = 0.5 * (zscat[grain, i, 1:n_theta, 0] + zscat[grain, i, 0:n_theta - 1, 0])
            dum = 0.5 * zav * dmu
            sum = dum.sum() * 4 * np.pi
            k_sca[grain, i] = sum

            mu_2 = 0.5 * (np.cos(theta[1:n_theta] * np.pi / 180.) + np.cos(theta[0:n_theta - 1] * np.pi / 180.))
            P_mu = 0.5 * ((2 * np.pi * zscat[grain, i, 1:n_theta, 0] / k_sca[grain, i]) + (2 * np.pi * zscat[grain, i, 0:n_theta - 1, 0] / k_sca[grain, i]))
            g[grain, i] = np.sum(P_mu * mu_2 * dmu)

opac_dict['zscat'] = zscat

Plot an example of a phase function before and after the chopping

In [ ]:
i_grain = a_opac.searchsorted(0.01 * 2 * np.pi)
i_lam = lam_opac.searchsorted(1e-4)

_ang = opac_dict['theta'] * np.pi / 180

f, ax = plt.subplots()
ax.semilogy(opac_dict['theta'], zscat[i_grain, i_lam, :, 0], label='Z11 (chopped)');
ax.semilogy(opac_dict['theta'], zscat_nochop[i_grain, i_lam, :, 0], label='Z11 (original)', ls='--');
ax.set_xlim(0, 50)
ax.legend();

## Emcee part

here we define some inputs and initial parameter sets for the optimization

In [ ]:
# defining number of walkers
nwalkers = 25
ndim     = 7

# setting the priors for some parameters instead of letting them be uniform randoms between (0.1)

sigma_coeff_0   = 10**((np.random.rand(nwalkers)-0.5)*4)
others_0        = np.random.rand(ndim-3,nwalkers)
d2g_coeff_0     = (np.random.rand(nwalkers)+0.5) / 100
d2g_exp_0       = (np.random.rand(nwalkers)-0.5) 

# the input matrix of priors
p0 = np.vstack((sigma_coeff_0,others_0, d2g_coeff_0, d2g_exp_0)).T

# logprob testing

In [ ]:
parameters = p0[0, :]
# The different indices in the parameters list correspond to different physical paramters
sigma_coeff = parameters[0]
sigma_exp = parameters[1]
size_exp = parameters[2]
amax_coeff = parameters[3]
amax_exp = parameters[4]
d2g_coeff = parameters[5]
d2g_exp = parameters[6]

create a temporary folder in the current folder

In [ ]:
temp_directory = tempfile.TemporaryDirectory(dir='.')
temp_path = temp_directory.name

set some disk specific parameters (the commented-out values are the ones that were used before)

In [ ]:
# mstar = 0.7 * MS
mstar = 10.**dh.sources.loc[disk]['log M_star/M_sun'] * M_sun

# lstar = 1.56 * LS
lstar = 10.**dh.sources.loc[disk]['log L_star/L_sun'] * L_sun

# tstar = 4266.00
tstar = 10.**dh.sources.loc[disk]['log T_eff/ K']

rstar = np.sqrt(lstar / (4 * np.pi * c.sigma_sb.cgs.value * tstar**4))

nr = 100
rin = 0.1 * au
r_c = 300 * au  # ??
rout = 400 * au  # 400au from avenhaus paper  #DSHARP Huang 2018 says 290 au
alpha = 1e-3

In [ ]:
PA = dh.sources.loc[disk]['PA']
inc = dh.sources.loc[disk]['inc']
dpc = dh.sources.loc[disk]['distance [pc]']

### make the disklab 2D model

In [ ]:
opac_params = ['dustcomponents', {'method': 'simplemixing'}]

d = disklab.DiskRadialModel(mstar=mstar, lstar=lstar, tstar=tstar, nr=nr, alpha=alpha, rin=rin, rout=rout)

d.make_disk_from_simplified_lbp(sigma_coeff, r_c, sigma_exp)

if d.mass / mstar > 0.2:
    #return -np.inf
    warnings.warn('Disk mass is unreasonably high: M_disk / Mstar = {d.mass/mstar:.2g}')

In [ ]:
d2g = d2g_coeff * ((d.r / au)**d2g_exp)
a_max = amax_coeff * (d.r/au)**(-amax_exp)

In [ ]:
a_i = get_interfaces_from_log_cell_centers(a_opac)
a, a_i, sig_da = get_powerlaw_dust_distribution(d.sigma * d2g, np.minimum(a_opac[-1], a_max), q=4 - size_exp, na=n_a, a0=a_i[0], a1=a_i[-1])

In [ ]:
f, ax = plt.subplots()

ax.contourf(d.r / au, a_opac, np.log10(sig_da.T))

ax.loglog(d.r / au, a_max, label='a_max')
ax.loglog(d.r / au, d2g, label='d2g')

ax.set_ylim(1e-5, 1e0)
ax.legend();

In [ ]:
for _sig, _a in zip(np.transpose(sig_da), a_opac):
    d.add_dust(agrain=_a, xigrain=rho_s, dtg=_sig / d.sigma)

In [ ]:
# load the opacity from the previously calculated opacity table
for dust in d.dust:
    dust.grain.read_opacity(str(opac_fname))

In [ ]:
# compute the mean opacities
d.meanopacitymodel = opac_params
d.compute_mean_opacity()

In [ ]:
f, ax = plt.subplots()
ax.loglog(d.r / au, d.mean_opacity_planck)
ax.loglog(d.r / au, d.mean_opacity_rosseland)

def movingaverage(interval, window_size):
    window = np.ones(int(window_size)) / float(window_size)
    return np.convolve(interval, window, 'same')

d.mean_opacity_planck[7:-7] = movingaverage(d.mean_opacity_planck, 10)[7:-7]
d.mean_opacity_rosseland[7:-7] = movingaverage(d.mean_opacity_rosseland, 10)[7:-7]

ax.loglog(d.r / au, d.mean_opacity_planck, 'C0--')
ax.loglog(d.r / au, d.mean_opacity_rosseland, 'C1--')

In [ ]:
f, ax = plt.subplots()
ax.loglog(d.r / au, d.tmid)

Make a 2D model out of it

In [ ]:
for iter in range(100):
    d.compute_hsurf()
    d.compute_flareindex()
    d.compute_flareangle_from_flareindex(inclrstar=True)
    d.compute_disktmid(keeptvisc=False)
    d.compute_cs_and_hp()

disk2d = disklab.Disk2D(
    disk=d,
    meanopacitymodel=d.meanopacitymodel,
    nz=100)

# snippet vertstruc 2d_1
for vert in disk2d.verts:
    vert.iterate_vertical_structure()
disk2d.radial_raytrace()
for vert in disk2d.verts:
    vert.solve_vert_rad_diffusion()
    vert.tgas = (vert.tgas**4 + 15**4)**(1 / 4)
    for dust in vert.dust:
        dust.compute_settling_mixing_equilibrium()

rmcd = disklab.radmc3d.get_radmc3d_arrays(disk2d, showplots=False)

In [ ]:
# Assign the radmc3d data

# nphi = rmcd['nphi']
ri = rmcd['ri']
thetai = rmcd['thetai']
phii = rmcd['phii']
nr = rmcd['nr']
# nth = rmcd['nth']
# nphi = rmcd['nphi']
rho = rmcd['rho']

# we need to tile this for each species

rmcd_temp = rmcd['temp'][:, :, None] * np.ones(n_a)[None, None, :]

# Define the wavelength grid for the radiative transfer

lam_mic = lam_opac * 1e4

# Write the `RADMC3D` input

disklab.radmc3d.write_stars_input(d, lam_mic, path=temp_path)
disklab.radmc3d.write_grid(ri, thetai, phii, mirror=False, path=temp_path)
disklab.radmc3d.write_dust_density(rmcd_temp, fname='dust_temperature.dat', path=temp_path, mirror=False)
disklab.radmc3d.write_dust_density(rho, mirror=False, path=temp_path)
disklab.radmc3d.write_wavelength_micron(lam_mic, path=temp_path)
disklab.radmc3d.write_opacity(disk2d, path=temp_path)
disklab.radmc3d.write_radmc3d_input(
    {'scattering_mode': 5, 'scattering_mode_max': 5, 'nphot': 10000000},
    path=temp_path)

calculate the mm continuum image

In [ ]:
fname_mm_sim = Path(temp_path) / 'image.fits'
lam_obs = 0.125
rd = 300 * au
disklab.radmc3d.radmc3d(
    f'image incl 47.5 posang -144.4 npix 500 lambda {lam_obs * 1e4} sizeau {4 * rd / au} secondorder  setthreads 1',
    path=temp_path)

In [ ]:
im = disklab.radmc3d.read_image(filename=str(fname_mm_sim.with_suffix('.out')))

In [ ]:
from radmc3dPy import image

In [ ]:
im = image.readImage(str(fname_mm_sim.with_suffix('.out')))
im.writeFits(str(fname_mm_sim), dpc=dpc, coord='15h56m09.17658s -37d56m06.1193s')

In [ ]:
data_mm_sim = imagecube(str(fname_mm_sim), clip=2.5)

x_as_sim, y_sim, dy_sim = data_mm_sim.radial_profile(inc=inc, PA=PA)

profile_mm_sim = (y_sim * u.Jy / data_mm_sim.beam_area_str).cgs.value
profile_mm_sim_err = (dy_sim * u.Jy / data_mm_sim.beam_area_str).cgs.value

In [ ]:
f, ax = plt.subplots()
ax.semilogy(x_as_obs, profile_mm_obs)
ax.fill_between(x_as_obs, profile_mm_obs - profile_mm_obs_err, profile_mm_obs + profile_mm_obs_err, alpha=0.5)

ax.semilogy(x_as_sim, profile_mm_sim)
ax.fill_between(x_as_sim, profile_mm_sim - profile_mm_sim_err, profile_mm_sim + profile_mm_sim_err, alpha=0.5)

ax.set_ylim(bottom=1e-16);

In [ ]:
f, ax = plt.subplots()
ax.imshow(np.log10(im.image[:, :, 0]))

In [ ]:
# obtaining the resultant radial intensity profile and subsequent calculation
# of the logprob from this image divided by number of observation

radial_profile = []
radial = []
for x, y in zip(range(0, 251, 1), range(249, 500, 1)):
    radial_profile.append(
        im.image[y][int(round(249 + x * np.tan(0.6213372)))])
    radial.append(np.sqrt(
        (im.x[int(round(249 + x * np.tan(0.6213372)))] / au)**2 + (im.y[y] / au)**2))
radial = np.asarray(radial)
radial_profile = np.asarray(radial_profile)

radial_profile = radial_profile[radial > 158]
radial = radial[radial > 158]

observed_intensity = observed_intensity[observed_radius > 1]
observed_intensity_error = observed_intensity_error[observed_radius > 1]
observed_radius = observed_radius[observed_radius > 1]

In [ ]:
log_prob_mm = -0.5 * np.sum((np.interp(observed_radius, radial / 158,
                                       radial_profile) - observed_intensity)**2 / (observed_intensity_error**2)) / len(observed_radius)

# Scattered light

To deproject the scattered light image, we will need to know where the scattering surface is. This is based on the Avenhaus et al. 2018 paper. In `imagecube` this surface can be defined with `z0` and `psi` such that its height $z$ is at

$\mathsf{z = z0 \, \left(\frac{r}{arcsec}\right)^{psi}}\, arcsec$

In [ ]:
z0 = 0.2
psi = 1.27

In [ ]:
# CUT OUT OPACITIES PART 2
from disklab.radmc3d import write

for p in Path(temp_path).glob('dustkappa_*.inp'):
    p.unlink()

for i_grain in range(n_a):
    opacity.write_radmc3d_scatmat_file(i_grain, opac_dict, f'{i_grain}', path=temp_path)

with open(Path(temp_path) / 'dustopac.inp', 'w') as f:
    write(f, '2               Format number of this file')
    write(f, '{}              Nr of dust species'.format(len(agrains)))

    for x in agrains:
        i_grain = agrains.searchsorted(x)
        write(f, '============================================================================')
        write(f, '10               Way in which this dust species is read')
        write(f, '0               0=Thermal grain')
        write(f, '{}              Extension of name of dustscatmat_***.inp file'.format(i_grain))

    write(f, '----------------------------------------------------------------------------')

# image calculation
rd = 250 * au
radmc3d.radmc3d(f'image incl {inc} posang {PA-90} npix 500 lambda 1.65 sizeau 1000 setthreads 4', path=temp_path)

fname_sca_img = Path(temp_path) / 'image.fits'
im = image.readImage(fname_sca_img.with_suffix('.out'))
im.writeFits(fname_sca_img, dpc=dpc, coord='15h56m09.17658s -37d56m06.1193s')

**TODO**

- check image brightness conversion of the sphere image

### read scattered light observations

In [ ]:
fname_sca_obs = 'Qphi_IMLup.fits'

clip = 3.0 # at how many arcsec to clip the image data

# fix the header of the sphere image
hdulist = fits.open(fname_sca_obs)
hdu0 = hdulist[0]

hdu0.header['cdelt1'] = -3.405e-06
hdu0.header['cdelt2'] = 3.405e-06
hdu0.header['crpix1'] = hdu0.header['naxis1'] // 2 + 1
hdu0.header['crpix2'] = hdu0.header['naxis2'] // 2 + 1
hdu0.header['crval1'] = 0.0
hdu0.header['crval2'] = 0.0
hdu0.header['crval3'] = 1.65e-4

# read it with imagecube and derive a radial profile

data = imagecube(hdulist, clip=clip)

x, y, dy = data.radial_profile(inc=inc, PA=PA, z0=z0, psi=psi)

# convert from Jy / beam to cgs

profile_sca = (y * u.Jy / data.beam_area_str).cgs.value
profile_sca_err = (dy * u.Jy / data.beam_area_str).cgs.value

### read scattered light RADMC3D image

In [ ]:
# calculating the profile from scattered light image using imagecube
sim_data = imagecube(fname_sca_img, clip=clip)

sim_x, sim_y, sim_dy = sim_data.radial_profile(inc=inc, PA=PA, z0=z0, psi=psi)

sim_profile = (sim_y * u.Jy / sim_data.beam_area_str).cgs.value
sim_profile_err = (sim_dy * u.Jy / sim_data.beam_area_str).cgs.value

profile = profile[x > 1]
profile_err = profile_err[x > 1]
x = x[x > 1]

sim_profile = sim_profile[sim_x > 1]
sim_profile_err = sim_profile_err[sim_x > 1]
sim_x = sim_x[sim_x > 1]

### calculate $\log P$

In [ ]:
log_prob_scat = -0.5 * np.nansum((np.interp(x, sim_x, sim_profile) - profile)**2 / (profile_err**2)) / len(x)

# adding the two log probs and then multiplying by a large factor in order to make the MCMC more sensitive to changes
log_prob = (log_prob_mm + log_prob_scat)

<hr>

**Here comes the rest of `MCMC_parallelized.py`, not cleaned up yet**

In [ ]:
print('step1')

# Parallelizing the simluation and running it for 250 iterations
with Pool(processes=100) as pool:
    sampler1 = emcee.EnsembleSampler(nwalkers, ndim, logprob, args=[profile, profile_err, x_arcsec], pool=pool)
    sampler1.run_mcmc(p0, 250)

print(sampler1.iteration)    

print('step2')
sampler2 = deepcopy(sampler1)
sampler2.log_prob_fn = None
with open('sampler.pickle', 'wb') as fid:
    pickle.dump(sampler2, fid)